In [1]:
import time
import os
import sys

# Start time 
start = time.time()

# Define all necessary input variables

# define the EPSG code as a character string
# use the following link to change the EPSG code if you will work in a different region https://spatialreference.org/ref/epsg/
# EPSG code is a standardized numerical identifier for coordinate systems and spatial reference systems 
# used to locate geographic entities on the Earth's surface. 
# The EPSG code uniquely identifies a particular coordinate reference system (CRS), 
# which includes definitions for coordinate axes, units of measurement, and transformations from one CRS to another.

epsg_code = "EPSG:32651"

# set path directory where your data are kept
path = "C:/Users/mmama/Documents/COASTAL_WATER/Indo-Pacific_Seagrass/Satellite_data/TEST"

# name of the Satellite file you will be processing
sat_file = "Masinloc_PS_20230620_Nonnormalized"

wdir = os.path.join(path, sat_file)

# import seagrass library file which contains many Python code functions that this code depends on
# this seagrass_lib.py file should be stored in a sub-folder called "Python_library" which is in the folder path defined above

library_dir = os.path.join(path, "Python_library")
sys.path.insert(0, library_dir)
from seagrass_lib import *

# increase cache size to avoid memory constraints
from osgeo import gdal
gdal.SetCacheMax(2000000000)

# Open the shapefile that is stored in an ROIs subfolder under the Input_data subfolder, which is in your working directory
input_shp = os.path.join(wdir, "Input_data", "ROIs",  sat_file + ".shp")

shapefile = gpd.read_file(input_shp)

# Display the first few rows of the attribute table to see what the Classification ID name caterogies for your shapefile are
# these should reflect the different categories you drew in QGIS

# Get the column names
column_names = shapefile.columns

# Rename the first column of the shapefile to "Classname"
shapefile = shapefile.rename(columns={shapefile.columns[0]: "id"})

# View unique values in ID column
unique_values = shapefile["id"].unique()
print(unique_values)

# Define output file for newly renamed shapefile
output_shapefile_path = os.path.join(wdir, "Input_data", "ROIs",  sat_file + ".shp")

shapefile.to_file(output_shapefile_path)

# Now you will split multipart polygon into singlepart polygon, but first
# define where you will save the new singlepart polygon output
output_shp = os.path.join(wdir, "Input_data", "ROIs", sat_file + "_singlepart.shp") 

# Split multipart polygon into singlepart polygon
multipart_to_singlepart(shp_fp = input_shp, out_fp = output_shp)

satellite_scene = gdal.Open(os.path.join(wdir, 'Input_data', 'Rrs_image', sat_file + ".TIF"))
band_num = satellite_scene.RasterCount

# re-define the satellite scen you are working with
satellite_scene = os.path.join(wdir, 'Input_data', 'Rrs_image', sat_file + ".TIF")
input_image = satellite_scene

# define the shapefile to be the singlepart shapefile you created above
input_shp = os.path.join(wdir, "Input_data", "ROIs", sat_file + "_singlepart.shp") 

# pick a place to save the new output tiffs, it should be in the ROI subfolder
output_ROIs = os.path.join(wdir, "Input_data", "ROIs")

# Display the first few rows of the attribute table to see what the ID name caterogies for your shapefile are
# these should reflect the different categories you drew in QGIS

singlepart_shapefile = gpd.read_file(input_shp)

# View unique values in ID column
unique_values = singlepart_shapefile["id"].unique()
print(unique_values)

# extract the image spectra for each category ROI drawn in QGIS and save to the location you defined above
shp_to_roi(image_fp = input_image, output_dir = output_ROIs, shp_fp = input_shp, field_name = 'id')

#creat a folder to store the DCNN model, saved as a .h5 file
dcnn_fp = os.path.join(wdir, "Input_data", "DCNN_model", sat_file + ".h5")

#input data for the training are the tiffs of each ROI we extracted above
input_data = os.path.join(wdir, 'Input_data', 'ROIs') 

#Run the training code
image_classes = roi_classes(shp_fp = input_shp, field_name = 'id')

# Training 
train_dcnn(cnnFileName = dcnn_fp, epochs = 100 , training_data_directory = input_data, class_names = image_classes, numChannels = band_num, dimension = 3)
input_image = satellite_scene
classified_scene = 'classified_'+ sat_file
output_classification = os.path.join(wdir, "Classified_image", classified_scene + ".TIF")
dcnn_fp = os.path.join(wdir, "Input_data", "DCNN_model", sat_file + ".h5")
# Run the classification
dcnn_classification(image_fp = input_image, dcnn_fp = dcnn_fp, output_fp = output_classification)

# End time
end = time.time()
print(end - start)


['Seagrass' 'coral' 'Submerged_sand' 'Land' 'Deep_water' 'Turbid_water']
['Seagrass' 'coral' 'Submerged_sand' 'Land' 'Deep_water' 'Turbid_water']

KerasTensor(type_spec=TensorSpec(shape=(None, 3, 3, 32), dtype=tf.float32, name=None), name='dropout/Identity:0', description="created by layer 'dropout'")
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 1, 16), dtype=tf.float32, name=None), name='dropout_1/Identity:0', description="created by layer 'dropout_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 6), dtype=tf.float32, name=None), name='dense/Softmax:0', description="created by layer 'dense'")
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 3, 32)          288       
                                                                 
 dropout (Dropout)           (None, 3, 3, 32)          0         
                                          

C:\Users\mmama\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


82/82 [==============================] - 0s 3ms/step
803.7366745471954


<Figure size 640x480 with 0 Axes>